In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('pitches_final_simplified.csv', index_col = 0)

C:\ProgramData\Anaconda\envs\capstone\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [36]:
# List of players to go through for now
player_list = ['Clayton Kershaw', 'Justin Verlander', 'Max Scherzer', 'Stephen Strasburg', 'Gerrit Cole',
              'Jacob deGrom', 'Zack Greinke', 'Madison Bumgarner', 'Marcus Stroman', 'Chris Sale', 'Hyun-Jin Ryu']

In [39]:
# Create an empty df to hold all the pitch names
pitch_df = pd.DataFrame()
for name in player_list:
    kershaw = df[df['player_name']==name]

    # Drop intentional balls and pitch outs, they aren't real pitches and batters don't need to worry about them
    kershaw.drop(kershaw[kershaw['pitch_name']=='Intentional Ball'].index, inplace=True, errors = 'ignore')
    kershaw.drop(kershaw[kershaw['pitch_name']=='Pitch Out'].index, inplace=True, errors = 'ignore')
    kershaw.drop(kershaw[kershaw['pitch_name']=='Other'].index, inplace=True, errors = 'ignore')

    # Unable to classify/do regression with classes with only 1 instance
    # so drop the rows with that pitch as well as the row w/ corresponding 'prev_pitch'
    for i in range(len(kershaw['pitch_name'].value_counts())):
        if kershaw['pitch_name'].value_counts()[i] == 1:
            drop = kershaw['pitch_name'].value_counts().index[i]
            kershaw.drop(kershaw[kershaw['pitch_name']==drop].index, inplace=True)
            kershaw.drop(kershaw[kershaw['prev_pitch']==drop].index, inplace=True)
            
    # Mirror the transformation of the pitch names to dummies for use in TF model
    y = kershaw['pitch_name']
    pitch = pd.get_dummies(y.copy())
    # Append each dummie df's columns (pitch names) to the pitch_df
    # Must be turned into a pd.series to be appended
    pitch_df = pitch_df.append(pd.Series(pitch.columns.values), ignore_index = True)
    
# Add player names as the indices
pitch_df.index = player_list
# Save as a csv
pitch_df.to_csv('pitch_names.csv')

C:\ProgramData\Anaconda\envs\capstone\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [64]:
# Check it went accordingly
df = pd.read_csv('pitch_names.csv', index_col = 0)
df

,0,1,2,3
Clayton Kershaw,Breaking Ball,Fastball,Off-speed,NaN
Justin Verlander,Breaking Ball,Fastball,Off-speed,NaN
Max Scherzer,Breaking Ball,Fastball,Off-speed,NaN
Stephen Strasburg,Breaking Ball,Fastball,Off-speed,NaN
Gerrit Cole,Breaking Ball,Fastball,Off-speed,NaN
Jacob deGrom,Breaking Ball,Fastball,Off-speed,NaN
Zack Greinke,Breaking Ball,Fastball,Junk,Off-speed
Madison Bumgarner,Breaking Ball,Fastball,Off-speed,NaN
Marcus Stroman,Breaking Ball,Fastball,Off-speed,NaN
Chris Sale,Breaking Ball,Fastball,Off-speed,NaN


In [65]:
# Get the row values as a list
df[df.index == 'Clayton Kershaw'].dropna(axis = 1).values.tolist().pop()

['Breaking Ball', 'Fastball', 'Off-speed']